In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
 
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
 
drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pennylane
from IPython.display import clear_output
clear_output()

In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)
restart_runtime()

In [ ]:
# %matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import pennylane as qml
from pennylane import numpy as np
from pennylane import RX, RY, RZ, CNOT

In [ ]:
def feature_encoding_hamiltonian(features, wires):

    for idx, w in enumerate(wires):
        RX(features[idx], wires=w)

def ising_hamiltonian(weights, wires, l):

        # ZZ coupling
        CNOT(wires=[wires[1], wires[0]])
        RZ(weights[l, 0], wires=wires[0])
        CNOT(wires=[wires[1], wires[0]])
        # local fields
        for idx, w in enumerate(wires):
            RY(weights[l, idx + 1], wires=w)

def QAOAEmbedding(features, weights, wires):

    repeat = len(weights)
    for l in range(repeat):
        # apply alternating Hamiltonians
        feature_encoding_hamiltonian(features, wires)
        ising_hamiltonian(weights, wires, l)
    # repeat the feature encoding once more at the end
    feature_encoding_hamiltonian(features, wires)

In [ ]:
# load dataset

from numpy import loadtxt

label = 0

X = loadtxt('/content/drive/My Drive/Projects/quantum_image_classifier/Image Preprocessing/Dataset One-vs-All/x_train_2features_' + str(0) + '.txt', delimiter=',')

X_test = loadtxt('/content/drive/My Drive/Projects/quantum_image_classifier/Image Preprocessing/Dataset One-vs-All/x_test_2features_' + str(0) + '.txt', delimiter=',')

In [ ]:
Y = np.zeros((X.shape[0],))
Y[:int(X.shape[0]/2)] = 1
Y[int(X.shape[0]/2):] = -1

Y_test = np.zeros((X_test.shape[0],))
Y_test[:int(X_test.shape[0]/2)] = 1
Y_test[int(X_test.shape[0]/2):] = -1

In [ ]:
print(X.shape, Y.shape)
print(X_test.shape, Y_test.shape)

(504, 2) (504,)
(108, 2) (108,)


In [ ]:
A = X[:int(X.shape[0]/2)].copy()
B = X[int(X.shape[0]/2):].copy()

A_test = X_test[:int(X_test.shape[0]/2)].copy()
B_test = X_test[int(X_test.shape[0]/2):].copy()

print(A.shape, B.shape)
print(A_test.shape, B_test.shape)

(252, 2) (252, 2)
(54, 2) (54, 2)


In [ ]:
n_features = 2
n_qubits = 2 * n_features + 1

dev = qml.device("default.qubit", wires=n_qubits)

In [ ]:
@qml.qnode(dev)
def swap_test(q_weights, x1, x2):

    # load the two inputs into two different registers
    QAOAEmbedding(features=x1, weights=q_weights, wires=[1, 2])
    QAOAEmbedding(features=x2, weights=q_weights, wires=[3, 4])

    # perform the SWAP test
    qml.Hadamard(wires=0)
    for k in range(n_features):
        qml.CSWAP(wires=[0, k + 1, 2 + k + 1])
    qml.Hadamard(wires=0)

    return qml.expval(qml.PauliZ(0))

In [ ]:
def overlaps(weights, X1=None, X2=None):

    q_weights = weights

    overlap = 0
    for x1 in X1:
        for x2 in X2:
            # overlap of embedded intermediate features
            overlap += swap_test(q_weights, x1, x2)

    mean_overlap = overlap / (len(X1) * len(X2))
    return mean_overlap

In [ ]:
def cost(weights, A=None, B=None):

    aa = overlaps(weights, X1=A, X2=A)
    bb = overlaps(weights, X1=B, X2=B)
    ab = overlaps(weights, X1=A, X2=B)

    d_hs = -2 * ab + (aa + bb)

    return 1 - 0.5 * d_hs

In [ ]:
# generate initial parameters for circuit
init_pars_quantum = np.random.normal(loc=0, scale=0.1, size=(4, 3))

In [ ]:
optimizer = qml.RMSPropOptimizer(stepsize=0.01)
batch_size = 16
pars = init_pars_quantum

In [ ]:
#batch_size = 16
iter = 100

for i in range(iter):

    # Sample a batch of training inputs from each class
    selectA = np.random.choice(range(len(A)), size=(batch_size,), replace=True)
    selectB = np.random.choice(range(len(B)), size=(batch_size,), replace=True)
    A_batch = [A[s] for s in selectA]
    B_batch = [B[s] for s in selectB]

    # Walk one optimization step
    pars = optimizer.step(lambda w: cost(w, A=A_batch, B=B_batch), pars)
    print("Step", i, "done.")

    # Print the validation cost every 10 steps
    if i % 5 == 0 and i != 0:
        cst = cost(pars, A=A_test, B=B_test)
        print("Cost on validation set {:2f}".format(cst))

Step 0 done.
Step 1 done.
Step 2 done.
Step 3 done.
Step 4 done.
Step 5 done.
Cost on validation set 0.986210
Step 6 done.
Step 7 done.
Step 8 done.
Step 9 done.
Step 10 done.
Cost on validation set 0.986161
Step 11 done.
Step 12 done.
Step 13 done.
Step 14 done.
Step 15 done.
Cost on validation set 0.985556
Step 16 done.
Step 17 done.
Step 18 done.
Step 19 done.
Step 20 done.
Cost on validation set 0.984946
Step 21 done.
Step 22 done.
Step 23 done.
Step 24 done.
Step 25 done.
Cost on validation set 0.984603
Step 26 done.
Step 27 done.
Step 28 done.
Step 29 done.
Step 30 done.
Cost on validation set 0.983450
Step 31 done.
Step 32 done.
Step 33 done.
Step 34 done.
Step 35 done.
Cost on validation set 0.983175
Step 36 done.
Step 37 done.
Step 38 done.
Step 39 done.
Step 40 done.
Cost on validation set 0.982310
Step 41 done.
Step 42 done.
Step 43 done.
Step 44 done.
Step 45 done.
Cost on validation set 0.981612
Step 46 done.
Step 47 done.
Step 48 done.
Step 49 done.
Step 50 done.
Cost on 

In [ ]:
pars

NameError: ignored

In [ ]:
cost(pars, A=A_test, B=B_test)

0.9884888707724101